In [2]:
"""
papr-review-2023-02/notebook/oracle-manipulation.ipynb

Calculates the cost of attack to liquidate the highest LTV vault on PAPR
by manipulating the Uni V3 pool. Output is amount of PAPR needed to sell
into the Uni V3 PAPR/WETH spot pool referenced by the PAPR controller.

Ignores the PAPR controller cap on target-to-mark ratio.
"""

import typing as tp
import pandas as pd
import numpy as np
from ape import chain, Contract, networks

In [3]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [4]:
papr = Contract("0x320aAAB3038bc08317f5a4be19EA1D9608551d79")
papr

<PaprToken 0x320aAAB3038bc08317f5a4be19EA1D9608551d79>

In [5]:
pool = Contract("0x238cdFfc9097591D12F5C00136514FBE563f87BF")  # paprMEME/WETH 1% Uni V3 pool
pool

<UniswapV3Pool 0x238cdFfc9097591D12F5C00136514FBE563f87BF>

In [6]:
slot0 = pool.slot0()
slot0

slot0_return(sqrtPriceX96=78377225011399985617131305060, tick=-216, observationIndex=0, observationCardinality=1, observationCardinalityNext=1, feeProtocol=0, unlocked=True)

In [7]:
1.0001 ** slot0.tick  # looks good from papr.wtf UI

0.9786326662671399

In [8]:
controller = Contract("0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0")
controller

<PaprController 0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0>

In [9]:
controller.target() / 1e18  # looks good from papr.wtf UI

0.996140703515637

In [10]:
# check controller has same pool as ref
pool.address == controller.pool()

True

In [11]:
# some params we'll need from papr
block_now = chain.blocks.head
dt = chain.blocks.head.timestamp - controller.lastUpdated()
target = controller.target()
ltv_max = controller.maxLTV()
funding_period = controller.fundingPeriod()
cap_tm = int(3e18)  # internal constant

In [12]:
# print out papr params
print(f"PAPR params\ndt: {dt}\ntarget: {target}\nltv_max: {ltv_max}\nfunding_period: {funding_period}\ncap_tm: {cap_tm}")

PAPR params
dt: 112824
target: 996140703515637026
ltv_max: 500000000000000000
funding_period: 7776000
cap_tm: 3000000000000000000


In [13]:
deploy_block = 16592385  # block at which controller deployed

In [14]:
# find the PAPR vault with the highest LTV
#  1. gather all (account, nft) combos since deploy block
#  2. call vaultInfo view for each to get current status
%time query_debts = controller.IncreaseDebt.query('*', start_block=deploy_block)

INFO: Cache database has not been initialized
CPU times: user 8.26 s, sys: 1.05 s, total: 9.31 s
Wall time: 1min 6s


In [15]:
# utility functions to unfold event arguments into separate columns
def unfold_event_args(key: str, event_args: dict) -> tp.Any:
    return event_args[key]

def unfold(df: pd.DataFrame):
    if df.empty:
        return
    for key in df.iloc[-1]['event_arguments'].keys():
        df[key] = df['event_arguments'].apply(lambda ev: unfold_event_args(key, ev))

In [16]:
unfold(query_debts)

In [17]:
query_debts

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,account,collateralAddress,amount
0,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xbc3ed6B537f2980e66f396Fe14210A5...,0x5287d7960ff46f533dfe74840058fa6cb155433d75ef...,16592714,0x00c1594904a72f8fc22869e2e66149d975930145ec97...,514,177,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,11560000000000000
1,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xCC692D6E11268B40A1E3C58e3D86Fc4...,0x4cb9d0e1dbf432e6dd1875fc6d76718c5771b46159a3...,16592815,0x3afd3631f14f5664ccde758f7c788358f5d7606f2ba9...,298,71,0xCC692D6E11268B40A1E3C58e3D86Fc4CAAb9b77a,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,500000000000000000
2,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xbc3ed6B537f2980e66f396Fe14210A5...,0x8cb24f69414da14858d89d08cc46c03e32ff2d699b61...,16592963,0x25567545b30c47d5d858d3bff3b842c8fc54377b5cea...,431,204,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,11550000000000000
3,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0x82c1B61dA09b5FDce098a212Bb80702...,0xaac96e379eadd3ec5a634dff5832cd19e1f03050e6bd...,16592993,0xad684acb28b8631ea22c904d53141c888a7d97ff0f3b...,509,453,0x82c1B61dA09b5FDce098a212Bb8070210AB91049,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,446220000000000000
4,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xbc3ed6B537f2980e66f396Fe14210A5...,0xfb28b13ca4400c6028de057265b7466cd4c01d7e8eaf...,16593033,0xe1d5035cb57e00a9b6e2648915420dbc2be9c9f18cb8...,193,107,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,282510000000000000
...,...,...,...,...,...,...,...,...,...,...,...
106,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0x0e79c644eC160Afe87eA8e87201C0fe...,0x958230a5d1d80d2da3ad4f0af000a1c4de7197b2114a...,16735967,0xe179cf9218d849fa38d3f61cab90a76a69a29dddf09a...,279,159,0x0e79c644eC160Afe87eA8e87201C0fe35bA63E1f,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,469689466053851300
107,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0x01a746f9e48bd620b43FD1C5bAcADF8...,0xec66cd5c1add00d8918a281713f0137e4949ac6a9953...,16740850,0xbaba149bc11e0adfc2caff16d36e6c3661164619423e...,246,134,0x01a746f9e48bd620b43FD1C5bAcADF8474B41f46,0x5Af0D9827E0c53E4799BB226655A1de152A425a5,119080000000000000
108,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xE5501BC2B0Df6D0D7daAFC18D2ef127...,0x34d42c6fed5a0e4d7e1a5bbce404c02ee41748279bad...,16753015,0x7f958266849f09d69ec4294e4fe913cd480d6755be02...,61,40,0xE5501BC2B0Df6D0D7daAFC18D2ef127D9e612963,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,481700000000000000
109,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xE5501BC2B0Df6D0D7daAFC18D2ef127...,0x96176fe8dfb338d337208fcf75806d100fe896bd0936...,16753568,0xc233d602c66a490e38af5721b6a138e66d05b8f4275c...,110,69,0xE5501BC2B0Df6D0D7daAFC18D2ef127D9e612963,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,121140000000000000


In [18]:
# filter for unique (account, collateralAddress) combos
query_debts_grouped = query_debts.groupby(["account", "collateralAddress"]).size()
query_debts_grouped

account                                     collateralAddress                         
0x01a746f9e48bd620b43FD1C5bAcADF8474B41f46  0x5Af0D9827E0c53E4799BB226655A1de152A425a5    1
0x028B2b9adCa1BD1263086b0B221713A684968817  0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB    1
0x0EeDd9117f6b4B36be52F3368FD78392827C3E41  0x1A92f7381B9F03921564a437210bB9396471050C    1
0x0e79c644eC160Afe87eA8e87201C0fe35bA63E1f  0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42    1
0x13eEf4EF8FCa471f242ab0F8F49A3dB6017aDA33  0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD    1
                                                                                         ..
0xe70F46dB43085C4dA54Fe380ac04cB3955B50AB3  0x79FCDEF22feeD20eDDacbB2587640e45491b757f    1
0xe73B26fB7C84e4DE41703eD1A76EbF298638b57e  0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD    1
0xfa071DCf1FA4e777a9fAbFB6383Fb54e73B9D545  0x1CB1A5e65610AEFF2551A50f76a87a7d3fB649C6    1
                                            0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cd

In [19]:
vault_params = query_debts_grouped.index.tolist()
vault_params[:3]

[('0x01a746f9e48bd620b43FD1C5bAcADF8474B41f46',
  '0x5Af0D9827E0c53E4799BB226655A1de152A425a5'),
 ('0x028B2b9adCa1BD1263086b0B221713A684968817',
  '0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB'),
 ('0x0EeDd9117f6b4B36be52F3368FD78392827C3E41',
  '0x1A92f7381B9F03921564a437210bB9396471050C')]

In [20]:
# get all vaults
%time vaults = [{"account": params[0], "collateralAddress": params[1]} | controller.vaultInfo(*params).__dict__ for params in vault_params]

CPU times: user 1.48 s, sys: 93.5 ms, total: 1.57 s
Wall time: 25 s


In [21]:
vaults[:3]

[{'account': '0x01a746f9e48bd620b43FD1C5bAcADF8474B41f46',
  'collateralAddress': '0x5Af0D9827E0c53E4799BB226655A1de152A425a5',
  'count': 0,
  'auctionCount': 0,
  'latestAuctionStartTime': 0,
  'debt': 0},
 {'account': '0x028B2b9adCa1BD1263086b0B221713A684968817',
  'collateralAddress': '0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB',
  'count': 2,
  'auctionCount': 0,
  'latestAuctionStartTime': 0,
  'debt': 1258710000000000000},
 {'account': '0x0EeDd9117f6b4B36be52F3368FD78392827C3E41',
  'collateralAddress': '0x1A92f7381B9F03921564a437210bB9396471050C',
  'count': 0,
  'auctionCount': 0,
  'latestAuctionStartTime': 0,
  'debt': 0}]

In [22]:
# get the collateral values (in PAPR) for each vault
%time vaults = [ vault | {"collateral": int(vault["count"] * controller.cachedPriceForAsset(vault["collateralAddress"]).price * 1e18 / target)} for vault in vaults]

CPU times: user 1.44 s, sys: 96 ms, total: 1.53 s
Wall time: 23.8 s


In [23]:
# go to pandas df to make life easier
df_vaults = pd.DataFrame(data=vaults)

In [24]:
df_vaults = df_vaults[df_vaults["collateral"] > 0]  # only look at vaults with collateral and debt
df_vaults = df_vaults[df_vaults["debt"] > 0]

In [25]:
df_vaults["ltv"] = df_vaults["debt"] / df_vaults["collateral"]  # calc the ltvs for each

In [26]:
# sort by highest ltv
df_vaults_sorted = df_vaults.sort_values(by='ltv', ascending=False)
df_vaults_sorted

,account,collateralAddress,count,auctionCount,latestAuctionStartTime,debt,collateral,ltv
46,0xE5501BC2B0Df6D0D7daAFC18D2ef127D9e612963,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,32,0,0,1245090000000000000,3073174692230055424,0.405148
50,0xad387592c2De9645c3F42B5732B93833e312bD0C,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,535900000000000000,1339644212526832384,0.400032
58,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,1,0,0,220330000000000000,560111636871027136,0.393368
56,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,504460000000000000,1339644212526832384,0.376563
42,0xCC692D6E11268B40A1E3C58e3D86Fc4CAAb9b77a,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,500000000000000000,1339644212526832384,0.373233
53,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x1CB1A5e65610AEFF2551A50f76a87a7d3fB649C6,1,0,0,472020000000000000,1275677517759470080,0.370015
54,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,2,0,0,687320000000000000,1872254413709032192,0.367108
27,0x82c1B61dA09b5FDce098a212Bb8070210AB91049,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,484140000000000000,1339644212526832384,0.361394
52,0xbF8060106D2e83C106915A575BaeA3dc90c892a6,0xBd3531dA5CF5857e7CfAA92426877b022e612cf8,2,0,0,3581400000000000000,10197555389664430080,0.351202
1,0x028B2b9adCa1BD1263086b0B221713A684968817,0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB,2,0,0,1258710000000000000,3585263487283213312,0.351079


In [27]:
df_vaults_sorted['HF'] = (ltv_max / 1e18) / df_vaults_sorted['ltv']  # HF = LTV_max / LTV

In [28]:
df_vaults_sorted  # HF < 1 means liquidatable

,account,collateralAddress,count,auctionCount,latestAuctionStartTime,debt,collateral,ltv,HF
46,0xE5501BC2B0Df6D0D7daAFC18D2ef127D9e612963,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,32,0,0,1245090000000000000,3073174692230055424,0.405148,1.234117
50,0xad387592c2De9645c3F42B5732B93833e312bD0C,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,535900000000000000,1339644212526832384,0.400032,1.249901
58,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,1,0,0,220330000000000000,560111636871027136,0.393368,1.271074
56,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,504460000000000000,1339644212526832384,0.376563,1.327800
42,0xCC692D6E11268B40A1E3C58e3D86Fc4CAAb9b77a,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,500000000000000000,1339644212526832384,0.373233,1.339644
53,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x1CB1A5e65610AEFF2551A50f76a87a7d3fB649C6,1,0,0,472020000000000000,1275677517759470080,0.370015,1.351296
54,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,2,0,0,687320000000000000,1872254413709032192,0.367108,1.361996
27,0x82c1B61dA09b5FDce098a212Bb8070210AB91049,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,484140000000000000,1339644212526832384,0.361394,1.383530
52,0xbF8060106D2e83C106915A575BaeA3dc90c892a6,0xBd3531dA5CF5857e7CfAA92426877b022e612cf8,2,0,0,3581400000000000000,10197555389664430080,0.351202,1.423683
1,0x028B2b9adCa1BD1263086b0B221713A684968817,0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB,2,0,0,1258710000000000000,3585263487283213312,0.351079,1.424182


In [29]:
# calculate the liquidation mark tick (TWAT) for each vault:
# mark_tick_liq = 1/log(1.0001) * [ log(target) + (F/dt) * log(LTV / LTV_max) ]
def mark_tick_liq(ltv: float) -> int:
    base = 1.0001
    tick = (1 / np.log(base)) * (np.log(target / 1e18) + (funding_period / dt) * np.log(ltv / (ltv_max / 1e18)))
    return int(tick)

In [30]:
# worst vault right now has mark (TWAT) liq tick for one block manip of 1e-7 which is effectively zero
print('mark tick liq:', mark_tick_liq(df_vaults_sorted.iloc[0]['ltv']))
print('mark price liq:', 1.0001 ** mark_tick_liq(df_vaults_sorted.iloc[0]['ltv']))

mark tick liq: -145026
mark price liq: 5.034029378048701e-07


In [31]:
# but still above min tick price for uniswap pool
MIN_TICK = -887272
1.0001 ** MIN_TICK

2.938956807614301e-39

In [32]:
# if 10 bps within LTV max, then significantly easier to manipulate
print('mark tick liq:', mark_tick_liq(0.499))
print('mark price liq:', 1.0001 ** mark_tick_liq(0.499))

mark tick liq: -1418
mark price liq: 0.8678009503675866


In [33]:
# apply to ltvs for new column
df_vaults_sorted['twat_liq'] = df_vaults_sorted['ltv'].apply(lambda ltv: mark_tick_liq(ltv))

In [34]:
df_vaults_sorted

,account,collateralAddress,count,auctionCount,latestAuctionStartTime,debt,collateral,ltv,HF,twat_liq
46,0xE5501BC2B0Df6D0D7daAFC18D2ef127D9e612963,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,32,0,0,1245090000000000000,3073174692230055424,0.405148,1.234117,-145026
50,0xad387592c2De9645c3F42B5732B93833e312bD0C,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,535900000000000000,1339644212526832384,0.400032,1.249901,-153785
58,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,1,0,0,220330000000000000,560111636871027136,0.393368,1.271074,-165363
56,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,504460000000000000,1339644212526832384,0.376563,1.327800,-195457
42,0xCC692D6E11268B40A1E3C58e3D86Fc4CAAb9b77a,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,500000000000000000,1339644212526832384,0.373233,1.339644,-201578
53,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x1CB1A5e65610AEFF2551A50f76a87a7d3fB649C6,1,0,0,472020000000000000,1275677517759470080,0.370015,1.351296,-207547
54,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,2,0,0,687320000000000000,1872254413709032192,0.367108,1.361996,-212983
27,0x82c1B61dA09b5FDce098a212Bb8070210AB91049,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,484140000000000000,1339644212526832384,0.361394,1.383530,-223795
52,0xbF8060106D2e83C106915A575BaeA3dc90c892a6,0xBd3531dA5CF5857e7CfAA92426877b022e612cf8,2,0,0,3581400000000000000,10197555389664430080,0.351202,1.423683,-243514
1,0x028B2b9adCa1BD1263086b0B221713A684968817,0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB,2,0,0,1258710000000000000,3585263487283213312,0.351079,1.424182,-243755


In [35]:
beta = 12  # 12 second block times

In [36]:
# get the twat from the pool over last dt seconds by looking at cumulative tick values now vs dt seconds ago
# NOTE: papr does *not* use historical observations array for Uni pool so slot0.observationCardinality=1; instead papr effectively replicates Uni V2 cumulative tick calcs
tick_cum_now = pool.observe([0], block_identifier=block_now.number).tickCumulatives[0]
tick_cum_then = pool.observe([0], block_identifier=block_now.number - (dt // beta)).tickCumulatives[0]

In [37]:
# twat looks close to slot0.tick, which means calc seems right
twat = (tick_cum_now - tick_cum_then) / dt
twat

-218.6419910657307

In [38]:
slot0.tick

-216

In [39]:
# calculate the liquidation spot tick for each vault:
# spot_tick_liq = twat + (dt / beta) * (twat_liq - twat)
def spot_tick_liq(twat_liq: float) -> int:
    tick = twat + (dt / beta) * (twat_liq - twat)
    return int(tick)

In [40]:
# spot price to reach mark tick liq is out of range for uni pool tho when manip over 1 block
print('spot tick liq:', spot_tick_liq(df_vaults_sorted.iloc[0]['twat_liq']))
print('spot price liq:', 1.0001 ** spot_tick_liq(df_vaults_sorted.iloc[0]['twat_liq']))

spot tick liq: -1361478998
spot price liq: 0.0


In [41]:
# rapidly moves to within tick bounds and spot price of close to 1 given dt/beta is large (~ 1 day currently)
print("spot tick liq", spot_tick_liq(mark_tick_liq(0.499877)))
print("spot price liq", 1.0001 ** spot_tick_liq(mark_tick_liq(0.499877)))

spot tick liq 99837
spot price liq 21659.53058351105


In [42]:
# apply to twat_liqs for new column
df_vaults_sorted['tick_liq'] = df_vaults_sorted['twat_liq'].apply(lambda twat_liq: spot_tick_liq(twat_liq))

In [43]:
df_vaults_sorted

,account,collateralAddress,count,auctionCount,latestAuctionStartTime,debt,collateral,ltv,HF,twat_liq,tick_liq
46,0xE5501BC2B0Df6D0D7daAFC18D2ef127D9e612963,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,32,0,0,1245090000000000000,3073174692230055424,0.405148,1.234117,-145026,-1361478998
50,0xad387592c2De9645c3F42B5732B93833e312bD0C,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,535900000000000000,1339644212526832384,0.400032,1.249901,-153785,-1443831116
58,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,1,0,0,220330000000000000,560111636871027136,0.393368,1.271074,-165363,-1552687472
56,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,504460000000000000,1339644212526832384,0.376563,1.327800,-195457,-1835631260
42,0xCC692D6E11268B40A1E3C58e3D86Fc4CAAb9b77a,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,500000000000000000,1339644212526832384,0.373233,1.339644,-201578,-1893180902
53,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x1CB1A5e65610AEFF2551A50f76a87a7d3fB649C6,1,0,0,472020000000000000,1275677517759470080,0.370015,1.351296,-207547,-1949301440
54,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,2,0,0,687320000000000000,1872254413709032192,0.367108,1.361996,-212983,-2000410712
27,0x82c1B61dA09b5FDce098a212Bb8070210AB91049,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,484140000000000000,1339644212526832384,0.361394,1.383530,-223795,-2102065136
52,0xbF8060106D2e83C106915A575BaeA3dc90c892a6,0xBd3531dA5CF5857e7CfAA92426877b022e612cf8,2,0,0,3581400000000000000,10197555389664430080,0.351202,1.423683,-243514,-2287463174
1,0x028B2b9adCa1BD1263086b0B221713A684968817,0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB,2,0,0,1258710000000000000,3585263487283213312,0.351079,1.424182,-243755,-2289729056


In [44]:
# none of the existing vaults are liquidatable via oracle manip
df_vaults_sorted[df_vaults_sorted['tick_liq'] > MIN_TICK]

,account,collateralAddress,count,auctionCount,latestAuctionStartTime,debt,collateral,ltv,HF,twat_liq,tick_liq
